In [1]:
year = 2023
day = 23

In [2]:
from aocd import submit
from aocd.models import Puzzle
from functools import reduce
import numpy as np

puzzle = Puzzle(year=year, day=day)
data = puzzle.input_data
# data = puzzle.examples[0].input_data

data = data.strip()
data = data.split("\n")
data = [list(d) for d in data]
data = np.array(data)

printnp = lambda x: print(np.array2string(x, separator=' ', formatter={'str_kind': lambda x: x}))

In [3]:
import numpy as np

np.set_printoptions(edgeitems=30, linewidth=100000,
                    formatter=dict(float=lambda x: "%s" % x))

In [4]:
directions = [(0, 1), (0, -1), (1, 0), (-1, 0)]

def can_traverse(dy, dx, c):
    if c == ".": return True
    if c == ">" and (dy, dx) == (0, 1): return True
    if c == "<" and (dy, dx) == (0, -1): return True
    if c == "^" and (dy, dx) == (-1, 0): return True
    if c == "v" and (dy, dx) == (1, 0): return True
    return False

In [6]:
H, W = data.shape

goal = (H-1, W-2)
start = (0, 1)

out_of_bounds = lambda y, x: y < 0 or y >= H or x < 0 or x >= W
split_points = set()
dead_ends = 0
completed = []

# prev, curr, all
possible_paths = [(None, start, {start})]
for i in range(1, 5001):
    new_possible_paths = []
    for prev, curr, all in possible_paths:
        y,x = curr
        new_locs = []
        for dy, dx in directions:
            y_, x_ = (y + dy), (x + dx)
            if (y_, x_) == prev:
                continue
            if out_of_bounds(y_, x_):
                continue
            if (y_, x_) in all:
                continue
            if not can_traverse(dy, dx, data[y_, x_]):
                continue
            new_locs.append((y_, x_))

        if len(new_locs) == 1:
            if new_locs[0] == goal:
                completed.append((curr, all | {new_locs[0]}))
            else:
                new_possible_paths.append((curr, new_locs[0], all | {new_locs[0]}))
        elif len(new_locs) > 1:
            split_points.add(curr)
            for new_loc in new_locs:
                new_possible_paths.append((curr, new_loc, all | {new_loc}))
        elif len(new_locs) == 0:
            dead_ends += 1
    if i % 100 == 0:
        print(f"Possible paths: {len(possible_paths)}, completed: {len(completed)}, dead ends: {dead_ends}")
    possible_paths = new_possible_paths
    if len(possible_paths) == 0:
        break

Possible paths: 2, completed: 0, dead ends: 0
Possible paths: 2, completed: 0, dead ends: 0
Possible paths: 4, completed: 0, dead ends: 0
Possible paths: 6, completed: 0, dead ends: 0
Possible paths: 9, completed: 0, dead ends: 0
Possible paths: 17, completed: 0, dead ends: 0
Possible paths: 30, completed: 0, dead ends: 0
Possible paths: 46, completed: 0, dead ends: 0
Possible paths: 84, completed: 0, dead ends: 0
Possible paths: 129, completed: 0, dead ends: 0
Possible paths: 176, completed: 0, dead ends: 0
Possible paths: 215, completed: 0, dead ends: 0
Possible paths: 229, completed: 7, dead ends: 0
Possible paths: 218, completed: 26, dead ends: 0
Possible paths: 184, completed: 64, dead ends: 0
Possible paths: 128, completed: 123, dead ends: 0
Possible paths: 76, completed: 176, dead ends: 0
Possible paths: 46, completed: 206, dead ends: 0
Possible paths: 21, completed: 231, dead ends: 0
Possible paths: 13, completed: 239, dead ends: 0
Possible paths: 7, completed: 245, dead ends: 

In [7]:
answer = max([len(p[1])-1 for p in completed])
submit(answer, part="a", day=day, year=year)

Part a already solved with same answer: 2294


In [8]:
new_data = data.copy()
new_data[np.where(new_data == "<")] = "."
new_data[np.where(new_data == ">")] = "."
new_data[np.where(new_data == "^")] = "."
new_data[np.where(new_data == "v")] = "."

In [9]:
get_neighbours = lambda y, x: [(y + dy, x + dx) for dy, dx in directions if not out_of_bounds(y+dy, x+dx) and can_traverse(dy, dx, new_data[y + dy, x + dx])]

split_points = []
for y in range(H):
    for x in range(W):
        if new_data[y, x] != "#":
            if len(get_neighbours(y, x)) > 2:
                split_points.append((y, x))

split_points.append(start)
split_points.append(goal)

In [12]:
from collections import defaultdict
split_point_connections = defaultdict(lambda: defaultdict(int))
for sp in split_points:
    neighbours = get_neighbours(*sp)
    for n in neighbours:
        possible_paths = [(sp, n, [n])]
        for i in range(1, 5001):
            new_possible_paths = []
            for prev, curr, all in possible_paths:
                y,x = curr
                new_locs = []
                for dy, dx in directions:
                    y_, x_ = (y + dy), (x + dx)
                    if (y_, x_) == prev:
                        continue
                    if out_of_bounds(y_, x_):
                        continue
                    if (y_, x_) in all:
                        continue
                    if not can_traverse(dy, dx, new_data[y_, x_]):
                        continue
                    new_locs.append((y_, x_))

                if len(new_locs) == 1:
                    new_loc = new_locs[0]
                    if new_loc == goal or new_loc in split_points or new_loc == start:
                        split_point_connections[sp][new_loc] = len(all) + 1
                    else:
                        new_possible_paths.append((curr, new_loc, all + [new_loc]))
                elif len(new_locs) > 1:
                    raise ValueError(f"More than one neighbour {curr} {all}")
            possible_paths = new_possible_paths
            if len(possible_paths) == 0:
                break
        else:
            raise ValueError("No path found")

split_point_connections = {k: dict(v) for k, v in split_point_connections.items()}

In [13]:
curr = start
curr_cost = 0
path = [start]
possible_paths = [(curr, curr_cost, path)]
new_possible_paths = []
total_costs = []
for i in range(1000):
    print(i, f"Possible paths: {len(possible_paths)}")
    new_possible_paths = []
    for curr, curr_cost, path in possible_paths:
        if curr == goal:
            # print(f"Found path: {curr_cost} {path}")
            total_costs.append(curr_cost)
        for next, cost in split_point_connections[curr].items():
            if next in path:
                continue
            new_possible_paths.append((next, curr_cost + cost, path + [next]))
    possible_paths = new_possible_paths
    if len(possible_paths) == 0:
        break

0 Possible paths: 1
1 Possible paths: 1
2 Possible paths: 2
3 Possible paths: 4
4 Possible paths: 10
5 Possible paths: 24
6 Possible paths: 60
7 Possible paths: 146
8 Possible paths: 352
9 Possible paths: 816
10 Possible paths: 1812
11 Possible paths: 3812
12 Possible paths: 7834
13 Possible paths: 15068
14 Possible paths: 29306
15 Possible paths: 53692
16 Possible paths: 98656
17 Possible paths: 172462
18 Possible paths: 296710
19 Possible paths: 488322
20 Possible paths: 775104
21 Possible paths: 1175472
22 Possible paths: 1679102
23 Possible paths: 2291644
24 Possible paths: 2878738
25 Possible paths: 3452098
26 Possible paths: 3715688
27 Possible paths: 3761602
28 Possible paths: 3336758
29 Possible paths: 2683280
30 Possible paths: 1853774
31 Possible paths: 1074756
32 Possible paths: 521976
33 Possible paths: 166908
34 Possible paths: 43004
35 Possible paths: 1300


In [14]:
answer = max(total_costs)
submit(answer, part="b", day=day, year=year)

Part b already solved with same answer: 6418
